In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import random
import csv
import os
import math
import numpy as np
from tqdm import tqdm
from numba import jit
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None

In [ ]:
traindf = pd.read_pickle("./pickles/df_small_train.pkl")
# testdf = pd.read_pickle("./pickles/df_small_test.pkl")
# nep_testdf = pd.read_pickle("./pickles/df_small_train_test_yk.pkl")

# run for a very small dataset for testing
# traindf = pd.read_pickle("./pickles/df_verysmall_train_test_yk.pkl")


# run if you want to use the whole train/test dataset
# traindf = pd.read_csv("data/training_set_VU_DM.csv")
traindf.head()

In [ ]:
def competitors(df):
    """
    Make a new column in the dataframe (competitor_bool) for when there 
    exists a competitor and there are available rooms.
    1 is True, 0 is False.
    """
    competitor_bools = []
    availability_bools = []
    
    # Columns for existing competitors
    compare_cols = ["comp1_rate", "comp2_rate", "comp3_rate", "comp4_rate", "comp5_rate",
                "comp6_rate", "comp7_rate", "comp8_rate"]
    
    # Columns for room availability
    availability_cols = ["comp1_inv", "comp2_inv", "comp3_inv", "comp4_inv", "comp5_inv", 
                        "comp6_inv", "comp7_inv", "comp8_inv"]

    for index, row in df.iterrows():
        comp_bool = 0
        available = 0

        # Check for a combination of competitor price and room-availability. 
        for competitor, availability in zip(compare_cols, availability_cols):
            if row[competitor] == 1: # and row[availability] == 1: TODO!!!
                comp_bool = 1
            if row[availability] == 1:
                available = 1
        
        competitor_bools.append(comp_bool)
        availability_bools.append(available)
        
    # Append list of bools to new column
    df["competitor_bool"] = competitor_bools
    df["availability_bools"] = availability_bools
    
    return df

In [ ]:
def visitor_history(df):
    """
    Add column that tells us whether someone has visited a hotel before.
    Column name = total_visited; 1 is True, 0 is False.
    """
    
    # Get none-missing values
    hist_starrating = df.visitor_hist_starrating.isna()
    hist_adr = df.visitor_hist_adr_usd.isna()

    # Dit kan waarschijnlijk veel mooier en sneller maar het werkt...
    total_visited = []
    for index, row in df.iterrows():
        if hist_starrating[index] or hist_adr[index]:
            visited = 0
        else:
            visited = 1
        total_visited.append(visited)

    df["visited_before"] = total_visited
    
    return df

In [ ]:
def price_quality(df):
    """
    Add a column of ratio price/quality to the DataFrame.
    """
    price_quality = []
    
    # Check for missing values
    for index, row in df.iterrows():
        price, quality =  row["price_usd"],  row["prop_starrating"]
        if price and quality:
            ratio = price / quality
        else:
            ratio = None
        price_quality.append(ratio)
        
    df["price_quality"] = price_quality
    
    # Replace missing values with median
    df["price_quality"].fillna((df["price_quality"].median()), inplace=True)
    
    return df 

In [ ]:
def price_category(df):
    """
    Add a column of categories of price_usd and a column
    that corrected price for number of nights.
    Preprocessing of quantile cut showed that categories are:
    [(6.0889999999999995, 69.0] < (69.0, 90.0] < (90.0, 110.0] 
    < (110.0, 136.0] < (136.0, 170.077] < (170.077, 239.0] < (239.0, 554655.0]]
    
    """
    
    # Correct for number of nights ad add as new column
    df["price_correction"] = df["price_usd"] / df["srch_length_of_stay"]
    
    # Replace missing values
    df["price_correction"].fillna((df["price_correction"].median()), inplace=True)
    df["price_usd"].fillna((df["price_usd"].median()), inplace=True)
    
    # Make a new column of price categories
    df['PriceBand'] = pd.qcut(df["price_correction"], 7, labels=np.arange(1,8))
    
    return df

In [ ]:
def process_remaining_cols(df):
    """
    Add some remaining (and interesting columns) to the dataframe.
    """
    
    # Replace missing values
    df["promotion_flag"].fillna((df["promotion_flag"].mean()), inplace=True)
    df["prop_brand_bool"].fillna((df["prop_brand_bool"].median()), inplace=True)
    df["random_bool"].fillna((df["random_bool"].median()), inplace=True)
    
    return df

In [ ]:
def add_score(df):
    """
    Add a score 
    """
    
    # every hotel that is clicked on gets an importance score of 1
    df["importance"] = df["click_bool"]
    
    # every hotel that is booked gets an importance score of 5 
    df["importance"][df["booking_bool"] == 1] = 5
        
    return df


def preprocessing(traindf, dollarprice=True):
    traindf = competitors(traindf)
    print("cleaned competitors")
    
    traindf = visitor_history(traindf)
    print("cleaned visitor history")
    
    traindf = price_quality(traindf)
    print("cleaned price quality")
    
    traindf = price_category(traindf)
    print("cleaned price category")
    
    traindf = process_remaining_cols(traindf)
    print("did remaining columns")
    
    
    # Add relevant columns
    df = traindf[["prop_id", "srch_id", "position", "price_quality", 
                  "competitor_bool", "availability_bools", "visited_before", 
                  "click_bool", "booking_bool", "PriceBand", 
                  "promotion_flag", "prop_brand_bool", "random_bool"]]
        
    traindf = add_score(traindf)
    df.drop(columns=["click_bool", "booking_bool"])
    
    return traindf

In [ ]:
# Oh jit kan niet met pandas Series werken, moet omgeschreven worden naar np arrays
# @jit(nopython=True)
df = preprocessing(traindf)
# df.head(5)
df.describe()

In [ ]:
display(df.describe())

In [ ]:
df["price_quality"] = df.price_quality.astype(np.float32)
print(df.dtypes)
print()

if df.isnull().sum().sum() != 0:    
    print("\x1b[31mMissing values: \'\x1b[0m")
    print(df.isnull().sum())
else:
    print("\x1b[31mNo missing values!! :D \'\x1b[0m")

## Totale dataset zonder missende waardes, hier kan RandomForestClassifier op gerund worden

In [ ]:
filename = "./pickles/df_small_clean_test.pkl"
if not os.path.exists(filename):
    df.to_pickle(filename)

df.describe()

In [ ]:
y = df["prop_id"]
X = df.drop("prop_id", axis=1).copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
# Fit model on whole dataset
rfc = RandomForestClassifier(n_jobs=1)
model = rfc.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
X_test["predictions"] = predictions
X_test.head(10)

pred_groups = X_test.groupby('srch_id').agg({'position':lambda x: list(x)})
pred_groups

In [ ]:
rank_pred_groups   = X.groupby('srch_id').agg({'position':lambda x: list(x)})

# Get unique srchids to group by
unique_srchid = X.srch_id.unique()